In [11]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Connection details for source and target databases
server_source = 'DESKTOP-H1BEQI6'
database_source = 'AdventureWorks2022'
username_source = 'sa'
password_source = 'sa123456'
driver = 'ODBC Driver 17 for SQL Server'

# Connection details for data_test
server_target = 'DESKTOP-H1BEQI6'
database_target = 'HRdb'
username_target = 'sa'
password_target = 'sa123456'


# Create engine for source and target databases
connection_string_source = (
    f'mssql+pyodbc://{username_source}:{password_source}@{server_source}/{database_source}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_source = create_engine(connection_string_source, echo=False)

connection_string_target = (
    f'mssql+pyodbc://{username_target}:{password_target}@{server_target}/{database_target}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_target = create_engine(connection_string_target, echo=False)


## UNIQUNESS CHECK


In [27]:
Session = sessionmaker(bind=engine_target)
def get_tables(schema_name):
    """
    Retrieve all tables in the specified schema.

    :param schema_name: The schema name to retrieve tables from.
    :return: A list of table names in the specified schema.
    """
    query = f"""
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = :schema_name
    """

    try:
        with Session() as session:
            with session.begin():
                # Execute the query with the provided schema name
                result = session.execute(text(query), {'schema_name': schema_name})
                # Fetch all results and extract table names
                tables = [row['TABLE_NAME'] for row in result.mappings()]
        
        print(f"Tables in schema '{schema_name}':")
        print(tables)
        return tables
    
    except Exception as e:
        print(f"Error retrieving tables from schema '{schema_name}': {e}")
        return []

In [28]:

def get_columns(schema_name, table_name):
    """
    Retrieve all columns for the specified table in the schema, excluding XML columns.

    :param schema_name: The schema name where the table resides.
    :param table_name: The name of the table to retrieve columns from.
    :return: A list of column names in the specified table, excluding XML columns.
    """
    query = f"""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = :schema_name
      AND TABLE_NAME = :table_name
      AND DATA_TYPE <> 'xml'
    """

    try:
        with Session() as session:
            with session.begin():
                # Execute the query with the provided schema and table names
                result = session.execute(text(query), {'schema_name': schema_name, 'table_name': table_name})
                # Fetch all results and extract column names
                columns = [row['COLUMN_NAME'] for row in result.mappings()]
        
        print(f"Columns in table '{schema_name}.{table_name}':")
        print(columns)
        return columns
    
    except Exception as e:
        print(f"Error retrieving columns from table '{schema_name}.{table_name}': {e}")
        return []


In [17]:
def check_uniqueness(schema_name, table_name, unique_columns):
    """
    Check for uniqueness in the specified table based on the unique columns.
    
    :param schema_name: The schema name where the table resides.
    :param table_name: The name of the table to check.
    :param unique_columns: A list of columns that should have unique values.
    """
    if not unique_columns:
        print(f"No columns to check for uniqueness in {schema_name}.{table_name}.")
        return

    unique_columns_str = ', '.join(unique_columns)
    
    query = f"""
    SELECT {unique_columns_str}, COUNT(*) as count
    FROM {schema_name}.{table_name}
    GROUP BY {unique_columns_str}
    HAVING COUNT(*) > 1
    """
    
    try:
        with Session() as session:
            with session.begin():
                # Execute the query and load the result into a DataFrame
                result = session.execute(text(query))
                df_duplicates = pd.DataFrame(result.fetchall(), columns=result.keys())

        if not df_duplicates.empty:
            print(f"Duplicate records found in {schema_name}.{table_name} based on columns {unique_columns_str}:")
            print(df_duplicates)
        else:
            print(f"No duplicates found in {schema_name}.{table_name} based on columns {unique_columns_str}.")
    
    except Exception as e:
        print(f"Error checking uniqueness in {schema_name}.{table_name}: {e}")


In [18]:
schema_name = 'dimension'

# Get all tables from the schema
tables = get_tables(schema_name)

# Loop through each table and check columns
for table in tables:
    columns = get_columns(schema_name, table)
    if columns:
        check_uniqueness(schema_name, table, columns)


Tables in schema 'dimension':
['EmployeeDepartmentHistory', 'Employee_Payment', 'Personal_information', 'max_timestamp', 'Employeelogin', 'Department', 'Shift']
Columns in table 'dimension.EmployeeDepartmentHistory':
['EmployeeID', 'BusinessEntityID', 'ShiftID', 'StartDate', 'EndDate', 'ModifiedDate', 'JobTitle', 'IsCurrent', 'LoadTimestamp', 'Timestamp', 'scd_startdate', 'scd_enddate', 'ALT_shiftid']
No duplicates found in dimension.EmployeeDepartmentHistory based on columns EmployeeID, BusinessEntityID, ShiftID, StartDate, EndDate, ModifiedDate, JobTitle, IsCurrent, LoadTimestamp, Timestamp, scd_startdate, scd_enddate, ALT_shiftid.
Columns in table 'dimension.Employee_Payment':
['BusinessEntityID', 'RateChangeDate', 'Salary_hourly_rate', 'Salary_received_monthly', 'Salary_received_weekly', 'ModifiedDate', 'Inactive', 'Active', 'Hourly', 'Salaried']
No duplicates found in dimension.Employee_Payment based on columns BusinessEntityID, RateChangeDate, Salary_hourly_rate, Salary_received_

## Nulls Check

In [20]:
def check_nulls(schema_name, table_name):
    columns = get_columns(schema_name, table_name)
    if not columns:
        print(f"No columns found for table '{schema_name}.{table_name}'.")
        return
    
    null_counts = []
    
    for column in columns:
        query = f"""
        SELECT COUNT(*) as null_count
        FROM {schema_name}.{table_name}
        WHERE {column} IS NULL
        """
        
        try:
            with Session() as session:
                with session.begin():
                    result = session.execute(text(query))
                    null_count = result.scalar()  # Fetch the count of NULLs
                    null_counts.append((column, null_count))
        
        except Exception as e:
            print(f"Error checking NULLs in column '{column}' of table '{schema_name}.{table_name}': {e}")
    
    # Print results
    print(f"NULL values in table '{schema_name}.{table_name}':")
    for column, count in null_counts:
        print(f"Column '{column}': {count} NULL values")



In [21]:
# Example usage
schema_name = 'dimension'

# Get all tables from the schema
tables = get_tables(schema_name)

# Loop through each table and check for NULLs in columns
for table in tables:
    check_nulls(schema_name, table)

Tables in schema 'dimension':
['EmployeeDepartmentHistory', 'Employee_Payment', 'Personal_information', 'max_timestamp', 'Employeelogin', 'Department', 'Shift']
Columns in table 'dimension.EmployeeDepartmentHistory':
['EmployeeID', 'BusinessEntityID', 'ShiftID', 'StartDate', 'EndDate', 'ModifiedDate', 'JobTitle', 'IsCurrent', 'LoadTimestamp', 'Timestamp', 'scd_startdate', 'scd_enddate', 'ALT_shiftid']
NULL values in table 'dimension.EmployeeDepartmentHistory':
Column 'EmployeeID': 0 NULL values
Column 'BusinessEntityID': 0 NULL values
Column 'ShiftID': 0 NULL values
Column 'StartDate': 0 NULL values
Column 'EndDate': 298 NULL values
Column 'ModifiedDate': 0 NULL values
Column 'JobTitle': 0 NULL values
Column 'IsCurrent': 0 NULL values
Column 'LoadTimestamp': 0 NULL values
Column 'Timestamp': 0 NULL values
Column 'scd_startdate': 0 NULL values
Column 'scd_enddate': 298 NULL values
Column 'ALT_shiftid': 296 NULL values
Columns in table 'dimension.Employee_Payment':
['BusinessEntityID', '

## SALARY SUM CHECK

In [23]:
SourceSession = sessionmaker(bind=engine_source)

def get_sum_from_source(source_schema, source_table, column_name):
    query = f"""
    SELECT SUM({column_name}) AS total_sum
    FROM {source_schema}.{source_table}
    """
    
    try:
        with SourceSession() as session:
            result = session.execute(text(query))
            total_sum = result.scalar()
        
        print(f"Sum of '{column_name}' in source table '{source_schema}.{source_table}': {total_sum}")
        return total_sum
    
    except Exception as e:
        print(f"Error retrieving sum from source table '{source_schema}.{source_table}': {e}")
        return None

In [24]:
TargetSession = sessionmaker(bind=engine_target)
def get_sum_from_target(target_schema, target_table, target_column_name):
    query = f"""
    SELECT SUM({target_column_name}) AS total_sum
    FROM {target_schema}.{target_table}
    """
    
    try:
        with TargetSession() as session:
            result = session.execute(text(query))
            total_sum = result.scalar()
        
        print(f"Sum of '{target_column_name}' in target table '{target_schema}.{target_table}': {total_sum}")
        return total_sum
    
    except Exception as e:
        print(f"Error retrieving sum from target table '{target_schema}.{target_table}': {e}")
        return None


In [25]:
def compare_sums(source_schema, source_table, target_schema, target_table, target_column_name):
    
    source_sum = get_sum_from_source(source_schema, source_table, column_name)
    target_sum = get_sum_from_target(target_schema, target_table, target_column_name)
    
    if source_sum is not None and target_sum is not None:
        if source_sum == target_sum:
            print(f"Sums are equal for column '{column_name}': {source_sum}")
        else:
            print(f"Discrepancy found. Source sum: {source_sum}, Target sum: {target_sum}")

In [26]:
# Example usage
source_schema = 'HumanResources'
source_table = 'EmployeePayHistory'
column_name = 'Rate'  # Source column
target_schema = 'dimension'
target_table = 'Employee_Payment'
target_column_name = 'Salary_hourly_rate'  # Target column

compare_sums(source_schema, source_table, target_schema, target_table, target_column_name)

Sum of 'Rate' in source table 'HumanResources.EmployeePayHistory': 5611.7821
Sum of 'Salary_hourly_rate' in target table 'dimension.Employee_Payment': 5611.7821
Sums are equal for column 'Rate': 5611.7821


## National id check


In [29]:
Session = sessionmaker(bind=engine_target)

def count_invalid_national_ids(schema_name, table_name, column_name):
    query = f"""
    SELECT COUNT(*) AS invalid_count
    FROM {schema_name}.{table_name}
    WHERE LEN({column_name}) != 9 OR ISNUMERIC({column_name}) = 0
    """
    
    try:
        with Session() as session:
            result = session.execute(text(query))
            invalid_count = result.scalar()
        
        print(f"Count of invalid {column_name} entries in {schema_name}.{table_name}: {invalid_count}")
        return invalid_count
    
    except Exception as e:
        print(f"Error counting invalid {column_name} entries: {e}")
        return None

# Example usage
schema_name = 'dimension'
table_name = 'Personal_information'
column_name = 'NationalIDNumber'

count_invalid_national_ids(schema_name, table_name, column_name)

Count of invalid NationalIDNumber entries in dimension.Personal_information: 40


40